In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from pca import PCA

In [2]:
path = os.path.join(os.path.dirname(os.getcwd()), 'Data')
returns = pd.read_excel(os.path.join(path, 'Data.xlsx'), sheet_name='RETURNS').rename(columns={'Unnamed: 0': 'Date'})
returns = returns.set_index('Date')
returns.head()

,EUROSTOXX50,ANHEUSER-BUSCH INBEV SA/NV,KONINKLIJKE AHOLD DELHAIZE N,ADIDAS AG,AIR LIQUIDE SA,AIRBUS SE,ALLIANZ SE-REG,ASML HOLDING NV,BASF SE,BAYER AG-REG,...,SAFRAN SA,SANOFI,BANCO SANTANDER SA,SAP SE,SIEMENS AG-REG,SCHNEIDER ELECTRIC SE,TELEFONICA SA,TOTALENERGIES SE,VIVENDI SE,VOLKSWAGEN AG-PREF
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-31,0.032523,0.054383,0.003162,0.075629,0.041231,0.010972,0.036812,0.097633,0.002335,-0.026051,...,0.076236,0.019595,0.061884,0.100177,0.055098,0.141340,-0.011803,0.070689,-0.025699,0.091075
2006-02-28,0.022783,0.000543,0.077163,-0.046946,-0.021275,-0.046203,0.020732,-0.061456,-0.024685,-0.016879,...,-0.017479,-0.053677,0.042739,0.015130,0.025850,-0.002907,0.030260,-0.071304,-0.012797,0.219389
2006-03-31,0.021922,-0.002326,-0.051172,-0.005057,0.037439,0.130364,0.017577,-0.032165,0.022768,-0.021617,...,-0.020598,0.099439,-0.016324,0.046520,0.000520,0.039068,0.000768,0.031755,0.113553,0.047422
2006-04-28,-0.001103,0.045571,0.009242,0.025480,-0.001745,-0.100372,-0.036871,-0.002967,0.050386,0.107398,...,-0.027246,-0.047770,0.019916,-0.032454,-0.026485,0.007295,-0.019305,0.006891,0.021172,-0.028490
2006-05-31,-0.042522,-0.061532,-0.025948,-0.067838,-0.032199,-0.092904,-0.071193,-0.060119,-0.038437,-0.017612,...,-0.137736,-0.016054,-0.072645,-0.045965,-0.105059,-0.075689,0.025819,-0.048478,0.002404,-0.081681


In [3]:
stocks = returns.columns.tolist()
model = PCA(returns ,  stocks , 3 )
cov  = model.cov_matrix
eigen_values = model.eigenvalues
eigen_vectors = model.reduced_eigenvectors

eigen_vectors


,PC_1,PC_2,PC_3
EUROSTOXX50,0.212619,0.005017,-0.004756
ANHEUSER-BUSCH INBEV SA/NV,0.116523,-0.114423,-0.125037
KONINKLIJKE AHOLD DELHAIZE N,0.075654,-0.052626,-0.246661
ADIDAS AG,0.119877,-0.221344,0.036026
AIR LIQUIDE SA,0.145405,-0.127968,-0.151740
AIRBUS SE,0.141008,0.017677,0.025593
ALLIANZ SE-REG,0.179794,0.056574,0.083352
ASML HOLDING NV,0.124810,-0.161882,0.024809
BASF SE,0.173823,-0.082407,0.107706
BAYER AG-REG,0.134407,-0.071361,-0.066107


In [6]:
for i in stocks: 
    print(returns[i].head()) 

Date
2006-01-31    0.032523
2006-02-28    0.022783
2006-03-31    0.021922
2006-04-28   -0.001103
2006-05-31   -0.042522
Name: EUROSTOXX50, dtype: float64
Date
2006-01-31    0.054383
2006-02-28    0.000543
2006-03-31   -0.002326
2006-04-28    0.045571
2006-05-31   -0.061532
Name: ANHEUSER-BUSCH INBEV SA/NV, dtype: float64
Date
2006-01-31    0.003162
2006-02-28    0.077163
2006-03-31   -0.051172
2006-04-28    0.009242
2006-05-31   -0.025948
Name: KONINKLIJKE AHOLD DELHAIZE N, dtype: float64
Date
2006-01-31    0.075629
2006-02-28   -0.046946
2006-03-31   -0.005057
2006-04-28    0.025480
2006-05-31   -0.067838
Name: ADIDAS AG, dtype: float64
Date
2006-01-31    0.041231
2006-02-28   -0.021275
2006-03-31    0.037439
2006-04-28   -0.001745
2006-05-31   -0.032199
Name: AIR LIQUIDE SA, dtype: float64
Date
2006-01-31    0.010972
2006-02-28   -0.046203
2006-03-31    0.130364
2006-04-28   -0.100372
2006-05-31   -0.092904
Name: AIRBUS SE, dtype: float64
Date
2006-01-31    0.036812
2006-02-28    0.0

In [4]:
scores = model.reduced_pc_scores #PC1
scores

,PC_1,PC_2,PC_3
Date,,,
2006-01-31,3.181818,-1.534186,1.210880
2006-02-28,1.269649,0.435248,-0.341488
2006-03-31,1.794042,-0.319255,-0.729636
2006-04-28,-1.068337,-0.187204,0.180097
2006-05-31,-5.185730,1.540023,0.213714
...,...,...,...
2023-08-31,-2.196106,1.545860,-0.025831
2023-09-29,-2.968706,2.919038,0.051916
2023-10-31,-3.272628,1.481215,-0.689881


In [5]:
models = model.pca_model()

KeyError: 'EUROSTOXX50'